# Αναγνώριση ονοματισμένων ονοτήτων

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse
from spacy.scorer import Scorer



def data_preparation_for_NER(data_filename, test_size=0.25):
    """
    Given the file that contains annotated data for NER
    splits the dataset to train set and test set.

    Parameters
    ----------
    data_filename : str
        the path to the file that contains the annotated
        data for NER
        
    test_size : float
        the test_size that is given as a parameter to
        train_test_split function of scikit-learn

    Returns
    -------
    tuple
        the tuple of train set and test set
    """
    df_texts = pd.read_csv(data_filename, index_col=None, converters={'entities': eval})
    df_texts['text_entities'] = list(zip(df_texts.text, df_texts.entities))
    data = [(text, {'entities': entities}) for (text, entities) in df_texts['text_entities']]
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=10)
    return train_data, test_data



def train_ner(labels, train_set, model=None, new_model_name="", output_dir=None, n_iter=30):
    """
    Sets up the pipeline and named entity recognizer
    and trains the NER for new entities.

    Parameters
    ----------
    labels : list
        the labels (strings) of new entities 
        
    train_set : list
        the observations of the train set
        
    model : module
        existing SpaCy model
        
    new_model_name : str
        the name of the new model
        
    output_dir: str
        the directory of the output file where the new
        model will be saved
        
    n_iter : int
        the number of the iterations in the training process
    """
    random.seed(0)
    if model is not None:
        nlp = model.load()  # load existing spaCy model
    else:
        nlp = spacy.blank("el")  # create blank model in greek
    # add entity recognizer to model if it's not in the pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    for label in labels:
        ner.add_label(label)  # add new entity labels to entity recognizer
    
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(train_set)
            batches = minibatch(train_set, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses in iteration: " + str(itn + 1), losses)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        
        

def evaluate_ner(output_dir, test_set, print_entities=False):
    """
    Tests the new ner model and scores its performance.

    Parameters
    ----------
    output_dir: str
        the directory of the output file where the new
        model was saved
        
    test_set : list
        the observations of the test set
        
    print_entities : boolean
        if True the entities detected for each text
        in the test set will be printed
        
    Returns
    -------
    dict
        all the metrics ant their scores that measure
        the performance of the new NLP model
    """
    ner_model = spacy.load(output_dir)
    scorer = Scorer()
    # test ner
    for (input_, dict_entities) in test_set:        
        [annot] = dict_entities.values()
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        # precision, recall, f-score for ner etc.
        scorer.score(pred_value, gold)
        if print_entities:
            print("Entities in '%s'" % input_[:70])
            for ent in pred_value.ents:
                print(ent.label_, ent.text)
    return scorer.scores

In [4]:
train_data, test_data = data_preparation_for_NER('../data/proionta_tis_fisis_votana_annotated.csv', 0.25)
print('Training...')
train_ner(['ORGAN', 'PLANT'], train_data, new_model_name='ethnopharmacology_ner', output_dir='./ner_model/', n_iter=35)
print('Testing...')
scores = evaluate_ner('./ner_model/', test_data, True)
print(scores)

Training...
Losses in iteration: 1 {'ner': 32616.148113556148}
Losses in iteration: 2 {'ner': 10389.246237399871}
Losses in iteration: 3 {'ner': 6961.101458436975}
Losses in iteration: 4 {'ner': 5019.090962342684}
Losses in iteration: 5 {'ner': 3700.6688318366473}
Losses in iteration: 6 {'ner': 2993.1306144419123}
Losses in iteration: 7 {'ner': 2562.367896414356}
Losses in iteration: 8 {'ner': 2215.0164972063203}
Losses in iteration: 9 {'ner': 2032.6080897793313}
Losses in iteration: 10 {'ner': 1713.0978065025452}
Losses in iteration: 11 {'ner': 1524.5060370259166}
Losses in iteration: 12 {'ner': 1477.710753643839}
Losses in iteration: 13 {'ner': 1302.4753524607952}
Losses in iteration: 14 {'ner': 1205.9807149715134}
Losses in iteration: 15 {'ner': 1114.7112169968445}
Losses in iteration: 16 {'ner': 1064.856284377494}
Losses in iteration: 17 {'ner': 1015.2632681475288}
Losses in iteration: 18 {'ner': 951.7646556733364}
Losses in iteration: 19 {'ner': 854.7619009003638}
Losses in iterat

Entities in '  Βότανα  Σμύρνα, η ιερή!ΒόταναΣμύρνα, η ιερή!από Η τροφή μας το φάρμα'
PLANT νεογέννητο
PLANT μύρρο
PLANT μύρρο
PLANT ψύλλους
PLANT μύρρου
ORGAN γλώσσα
ORGAN βλεννογόνων
ORGAN λάρυγγα
PLANT μύρρου
PLANT Μύρρο
Entities in '  Βότανα  Gotu Kola-Σεντέλα, ενεργοποιεί νευρικό σύστημα και εγκέφαλο,'
ORGAN εγκέφαλο
ORGAN εγκέφαλο
ORGAN εγκέφαλο
ORGAN εγκεφάλου
ORGAN εγκέφαλο
ORGAN εγκεφάλου
ORGAN φλέβες
ORGAN δέρματος
ORGAN επινεφριδίων
ORGAN φλεβών
ORGAN τριχοειδών αγγείων
ORGAN φλέβες
ORGAN αιμοφόρων αγγείων
ORGAN μάτια
ORGAN δέρμα
PLANT Κισσού
ORGAN πνεύμονες
ORGAN στομάχου
PLANT κρεμμύδια
PLANT λεμονιού
PLANT τσίλι
PLANT καρύδας
ORGAN μυαλό
PLANT πιπεριού
PLANT τζίντζερ
ORGAN εγκεφάλου
ORGAN συκώτι
ORGAN εγκέφαλο
Entities in '  Βότανα  Ραδίκια: Ποικιλίες, Θεραπευτικές ιδιότητες, Αγορά-ΣυντήρησηΒ'
PLANT Ραδίκια
PLANT Ραδίκια
PLANT Ραδίκια
PLANT Ραδίκια
PLANT Ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδίκι
PLANT Ταραξάκο


Entities in '  Βότανα  Τα καλύτερα φυσικά μυοχαλαρωτικά βόταναΒόταναΤα καλύτερα φυσ'
ORGAN μυς
ORGAN μυς
ORGAN μυς
ORGAN μυς
ORGAN εγκέφαλο
ORGAN στομάχι
ORGAN αρθρώσεις
PLANT Βαλεριάνα
PLANT βαλεριάνα
PLANT βαλεριάνας
PLANT Βαλεριάνα
ORGAN νεύρα
ORGAN μυς
PLANT Μαγιοβότανο
PLANT νεπέτα
PLANT μέντας
ORGAN αρθρώσεις
PLANT Μαγιοβότανο
PLANT νεπέτα
PLANT Πιπέρι
ORGAN έντερο
PLANT ραπανάκια
PLANT θυμάρι
PLANT θυμάρι
ORGAN μυς
PLANT Θυμάρι
PLANT Πασιφλόρα
PLANT Πασιφλόρα
PLANT Λεβάντα
PLANT λεβάντας
ORGAN μυαλό
ORGAN μυς
PLANT Λεβάντα
PLANT Βαλσαμόχορτο
Entities in '  Βότανα  Γλυκάνισο - Pimpinella anisum: Άρωµα υγείαςΒόταναΓλυκάνισο –'
PLANT Γλυκάνισο
PLANT Γλυκάνισο
PLANT Γλυκάνισο
PLANT γλυκάνισο
PLANT γλυκάνισο
ORGAN νεύρα
PLANT γλυκάνισο
PLANT γλυκάνισου
ORGAN συκώτι
ORGAN μυών
ORGAN στομάχου
ORGAN εντέρων
ORGAN συκώτι
ORGAN εντέρων
PLANT γλυκάνισου
ORGAN πνεύμονες
PLANT γλυκάνισος
ORGAN στομάχου
ORGAN εντέρων
ORGAN εντέρων
ORGAN νεφρών
ORGAN συκωτιού
ORGAN παγκρέατος
PLANT δυόσμο
ORGA

Entities in '  Βότανα  Κρόκος και τζίνσενγκ απογειώνουν τη λίμπιντοΒόταναΦυσική Αντ'
PLANT Κρόκος
PLANT τζίνσενγκ
PLANT τζίνσενγκ
PLANT Κρόκος
PLANT τζίνσενγκ
PLANT κρόκο
PLANT τζίνσενγκ
ORGAN μυαλό
ORGAN εγκέφαλο
PLANT σκόρδο
PLANT σαλάμι
PLANT τσίλι
PLANT maca
PLANT τζίνσενγκ
PLANT σκόρδου
PLANT Τριβόλι
PLANT Ginseng
PLANT κρόκο
Entities in '  Βότανα  Ραδίκια (αντίδια) τα θαυματουργάΒόταναΛαχανικάΡαδίκια (αντίδ'
PLANT Ραδίκια
PLANT Ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT πικραλίδες
PLANT ταραξάκο
PLANT ραδίκια
PLANT βλίτα
PLANT γλιστρίδα
PLANT καυκαλήθρα
PLANT μάραθο
PLANT ραδίκι
ORGAN συκωτιού
PLANT ραδίκια
PLANT σταμναγκάθι
PLANT ραδίκι
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδίκια
PLANT ραδικιών
PLANT ραδίκια
PLANT ραδίκια
ORGAN ήπατος
PLANT κιχώριο
PLANT ραδίκι
ORGAN ήπατος
ORGAN χοληδόχου κύστεως
ORGAN στομάχου
PLANT τσουκνίδα
PLANT ραδίκια
PLANT ραδικιού
PLANT λεμόνι
PLANT Ταραξάκο
PLANT πικραλίδα
Entities in '  Βότανα  Η εκδίκηση της... φραπελιάς;ΒόταναΕλλάδαΗ εκδί

Entities in '  Βότανα  Δεντρολίβανο-Rosmarinus officinalis για τη μνήμηΒόταναΠρόληψ'
ORGAN μυαλό
ORGAN εγκέφαλο
ORGAN εγκέφαλο
PLANT κύριου
PLANT σωσίβια
ORGAN εγκεφάλου
ORGAN εγκεφάλου
ORGAN εγκέφαλο
ORGAN αυτί
ORGAN συκώτι
PLANT ρίγανη
Entities in '  Βότανα  Αρχαίοι Έλληνες και βότανα!ΒόταναΕλλάδαΑρχαίοι Έλληνες και β'
ORGAN λεπτομερέστατες
ORGAN αιγός
PLANT μανδραγόρα
PLANT ελλέβορο
PLANT κατευναστικόν
PLANT μανδραγόρα
PLANT Κρότωνα
PLANT ελλέβορος
PLANT ελλέβορος
PLANT ιπποφαές
PLANT σίλφιο
PLANT κνίκος
PLANT πράσο
PLANT σέλινο
PLANT Άγνος
PLANT ύσσωπος
PLANT μύρρα
PLANT ρόδια
PLANT κυδώνια
PLANT ελλέβορος
PLANT ελλέβορος
PLANT κράμβης
PLANT λάπαθο
PLANT αλκυόνειο
PLANT σέλινο
PLANT ράμνος
PLANT κύμινο
PLANT κηρόπισσος
PLANT μύρρα
PLANT μαστίχα
PLANT βελανιδιάς
PLANT πεύκο
PLANT μύρρα
PLANT κρόκος
PLANT Nigella
PLANT μάραθο
PLANT κύμινο
PLANT μύρρα
PLANT κρόκος
PLANT κασσία
PLANT Ερασίστρατο
Entities in '  Βότανα  Ιπέροξο-Lapacho-Pau d’ Arco: Το θεραπευτικό βότανο των ΙΝΚΑΣ'
PLANT 

...

Entities in '  Βότανα  Θυμάρι: Ιδιότητες και χρήσεις ανά τους αιώνεςΒόταναΘυμάρι: Ι'
PLANT Θυμάρι
PLANT Θυμάρι
PLANT Θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT σύκων
PLANT θυμαριού
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
ORGAN λάρυγγα
PLANT θυμαριού
ORGAN γλώσσα
PLANT μαντζουράνα
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμάρι
PLANT θυμαριού
PLANT θυμάρι
PLANT θειαφιού
PLANT θυμάρι
PLANT θυμάρι
PLANT απήγανο
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμαριού
PLANT αρκεύθου
PLANT θυμαριού
PLANT θυμαριού
PLANT φασκόμηλου
PLANT θυμάρι
PLANT θυμάρι
PLANT Θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμάρι
PLANT σουσάμι
PLANT πέτρινο
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμάρι
PLANT θυμαριού
PLANT θυμάρι
PLANT Θυμάρι
PLANT Θυμάρι
PLANT Θυμάρι
PLANT θυμάρι
PLANT Θυμάρι
PLANT Θυμάρι
PLANT Θυμάρι
Entiti

Entities in '  Βότανα  Μέντα ή ασπιρίνη για ανακούφιση από τους πόνους;ΒόταναΕναλλα'
PLANT Μέντα
PLANT Μέντα
PLANT μέντας
PLANT μέντα
PLANT μέντα
PLANT δυόσμο
ORGAN αρθρώσεις
PLANT μέντας
PLANT δυόσμου
ORGAN δέρμα
PLANT μέντας
ORGAN δέρμα
ORGAN δέρμα
PLANT μέντα
PLANT Μέντα
PLANT δυόσμο
PLANT δυόσμος
ORGAN στομάχι
PLANT Σουσάμι
Entities in '  Βότανα  Θυμάρι: εξολοθρεύει τα μικρόβια σε λιγότερο από μία ώρα!!Βότ'
PLANT Θυμάρι
PLANT Θυμάρι
ORGAN πνευμόνων
PLANT θυμάρι
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
ORGAN εντέρου
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
ORGAN μυαλό
PLANT θυμαριού
PLANT θυμαριού
ORGAN ματιού
PLANT θυμαριού
ORGAN πνευμόνων
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
PLANT θυμαριού
PLANT λεβάντας
ORGAN μυών
PLANT θυμαριού
ORGAN νύχια
PLANT νυχιού
PLANT θυμάρι
ORGAN ουροδόχου κύστης
PLANT θυμάρι
ORGAN μήτρας
PLANT Θυμάρι
PLANT Θυμάρι
Entities in '  Βότανα  Άγριο χόρτο Ζοχός ή ζόχια: Ένα εκπληκτικό φ

Entities in '  Βότανα  Τσουκνίδα-urtica dioica, το βότανο πανάκειαΒόταναΤσουκνίδα-u'
PLANT Τσουκνίδα
PLANT Τσουκνίδα
PLANT Τσουκνίδα
PLANT τσουκνίδα
PLANT τσουκνίδα
PLANT τσουκνίδα
PLANT τσουκνίδας
ORGAN μύτης
PLANT μύρρο
PLANT τσουκνίδας
PLANT τσουκνίδα
PLANT καρότο
ORGAN συκώτι
PLANT τσουκνίδα
PLANT τσουκνίδας
PLANT τσουκνίδα
ORGAN δέρμα
ORGAN δέρμα
ORGAN δέρματος
PLANT μολόχα
PLANT μολόχα
PLANT τσουκνίδα
PLANT μολόχας
PLANT τσουκνίδας
PLANT τσουκνίδας
PLANT Μολόχα
PLANT τσουκνίδας
PLANT τσουκνίδα
PLANT τσουκνίδα
PLANT τσουκνίδες
PLANT τσουκνίδες
PLANT τσουκνίδες
PLANT τσουκνίδα
PLANT ΒΗΜΑΤΟΣ
PLANT τσουκνίδα
PLANT τσουκνίδα
PLANT τσουκνίδας
PLANT σπανάκι
PLANT λεμονιού
PLANT τσουκνίδα
ORGAN νεφρά
ORGAN στομάχου
ORGAN δέρματος
PLANT τσουκνίδα
PLANT τσουκνίδας
PLANT τείχη
PLANT τσουκνίδες
PLANT Τσουκνίδα
PLANT Τσουκνίδα
PLANT τσουκνίδας
Entities in '  Βότανα  Ασφόδελος: Θεραπευτικές Ιδιότητες, Χρήσεις και ΣυνταγέςΒόταν'
ORGAN βλεννογόνου
ORGAN στομάχου
ORGAN ήπατος
ORGAN μαστών
ORGAN ό

Entities in '  Βότανα  Προυνέλλα η κοινή: Αυτοεπούλωση για ανοιχτές ή χρόνιες πληγέ'
PLANT τριαντάφυλλα
ORGAN Καρδιά
ORGAN συκώτι
ORGAN χοληδόχο κύστη
ORGAN συκωτιού
ORGAN ματιών
ORGAN μήτρας
Entities in '  Αποτοξίνωσις  Ginkgo Biloba: Προστατεύει εγκέφαλο και όρχεις από τα '
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
PLANT Ginkgo Biloba
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo biloba
ORGAN εγκέφαλο
PLANT ginkgo biloba
PLANT Ginkgo biloba
PLANT Ginkgo biloba
PLANT Ginkgo Biloba
PLANT ginkgo
ORGAN εγκεφάλου
ORGAN όρχεων
PLANT Ginkgo biloba
ORGAN εγκέφαλο
ORGAN όρχεις
ORGAN εγκεφάλου
PLANT Ginkgo biloba
PLANT Ginkgo biloba
ORGAN εγκέφαλο
ORGAN όρχεις
ORGAN εγκέφαλο
ORGAN όρχεις
PLANT Ginkgo biloba
ORGAN εγκεφάλου
ORGAN όρχεων
ORGAN εγκεφάλου
ORGAN όρχεων
PLANT Ginkgo biloba
PLANT Ginkgo biloba
PLANT Ginkgo biloba
ORGAN εγκέφαλ

Entities in '  Βότανα  Ίταμος: Φαρμακευτικό δέντρο κατά του καρκίνου κινδυνεύει με '
PLANT Ίταμος
PLANT Ίταμος
PLANT Ίταμος
ORGAN μαστού
ORGAN ωοθηκών
PLANT πεύκα
PLANT πεύκα
PLANT πεύκων
ORGAN ωοθηκών
ORGAN μαστού
PLANT Ίταμος
PLANT εκπρόσωπος
PLANT ίταμος
PLANT ίταμος
PLANT ιτάμου
PLANT Ίταμος
PLANT ίταμου
PLANT ιτάμου
PLANT Λαβούρνο
PLANT ιτάμου
PLANT ιτάμου
PLANT ίταμος
PLANT ίταμου
ORGAN καρδιά
PLANT ιτάμου
ORGAN ωοθηκών
PLANT ίταμος
ORGAN ωοθηκών
ORGAN ήπατος
ORGAN πνεύμονα
PLANT Ίταμος
PLANT γκραβιόλα
PLANT ΓΚΡΑΒΙΌΛΑ
PLANT GRAVIOLA
PLANT κάνναβης
Entities in '  Βότανα  Βότανα και Μπαχαρικά: Πολύτιμη ασπίδα της υγείαςΒόταναΜπαχαρ'
PLANT κινόα
PLANT Πιπέρι
PLANT πιπέρι
PLANT πιπέρι
PLANT Μαϊντανός
PLANT μαϊντανός
PLANT Μαϊντανός
PLANT Ρίγανη
PLANT ρίγανη
PLANT ρίγανη
PLANT Γαρίφαλο
PLANT γαρίφαλο
PLANT γαρίφαλο
PLANT Κύμινο
PLANT κύμινο
PLANT Κύμινο
PLANT Τζίντζερ
PLANT τζίντζερ
ORGAN καρδιάς
ORGAN αρτηριών
PLANT τζίντζερ
PLANT Βασιλικός
PLANT βασιλικός
ORGAN εντέρου
PLANT Βασιλικ